In [1]:
%env WANDB_NOTEBOOK_NAME Train.ipynb
%env CUDA_DEVICE_ORDER PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES 1
%env PYTORCH_CUDA_ALLOC_CONF backend:cudaMallocAsync

env: WANDB_NOTEBOOK_NAME=Train.ipynb
env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=1
env: PYTORCH_CUDA_ALLOC_CONF=backend:cudaMallocAsync


In [2]:
from mp_20_utils import load_all_data
from tokenization import MASK_SITE
device = 'cuda'
dataset = 'mp_20_biternary'
datasets_pd, torch_datasets, site_to_ids, element_to_ids, spacegroup_to_ids, max_len = load_all_data(
    device=device, dataset=dataset)
print(max_len)

21


In [3]:
del torch_datasets['test'], datasets_pd['test']

In [4]:
from itertools import chain
from operator import itemgetter
max_enumeration = max(map(
    lambda dataframe: max(chain.from_iterable(dataframe.symmetry_sites_enumeration_padded)),
    datasets_pd.values()))
print(max_enumeration)

7


In [5]:
import torch
# UserWarning: TensorFloat32 tensor cores for float32 matrix multiplication available but not enabled. Consider setting `torch.set_float32_matmul_precision('high')` for better performance.
torch.set_float32_matmul_precision('high')
from wyckoff_transformer import WyckoffTransformerModel, WyckoffTrainer
d_hid = 170  # dimension of the feedforward network model in ``nn.TransformerEncoder``
nlayers = 3  # number of ``nn.TransformerEncoderLayer`` in ``nn.TransformerEncoder``
nhead = 1  # number of heads in ``nn.MultiheadAttention``
dropout = 0.1  # dropout probability
n_space_groups = len(spacegroup_to_ids)
# Not all 230 space groups are present in the data
model = WyckoffTransformerModel(
    n_space_groups = n_space_groups,
    n_sites = len(site_to_ids),
    n_elements = len(element_to_ids),
    max_enumeration = max_enumeration,
    d_space_groups = 16,
    d_sites = 64,
    d_species = 64,
    nhead=nhead,
    d_hid=d_hid,
    max_len=max_len,
    nlayers=nlayers,
    dropout=dropout).to(device)
#model = torch.compile(model)

In [6]:
import torch
trainer = WyckoffTrainer(model, torch_datasets, max_len,
                         site_mask=torch.tensor(site_to_ids[MASK_SITE]).to(device),
                         element_pad=torch.tensor(element_to_ids["PAD"]).to(device),
                         site_pad=torch.tensor(site_to_ids["PAD"]).to(device))
trainer.train(epochs=10000, val_period=10)

wandb: Currently logged in as: kazeev. Use `wandb login --relogin` to force relogin


wandb: Tracking run with wandb version 0.17.0


wandb: Run data is saved locally in /home/kna/WyckoffTransformer/wandb/run-20240517_150429-9pl53nyu
wandb: Run `wandb offline` to turn off syncing.


wandb: Syncing run fearless-wind-185


wandb: ⭐️ View project at https://wandb.ai/kazeev/WyckoffTransformer


...

Epoch 4660 val_loss_epoch 9.821868896484375 saved to checkpoints/2024-05-17_15-04-28/best_model_params.pt


wandb:                                                                                


wandb: 
wandb: Run history:
wandb:          known_seq_len ▅█▆▆▄▃▆██▁▄▃▅▇▃▅▂▂▂▇▅▃▆▅▄█▁▁█▁▆▆█▃▄█▃▇▂█
wandb:                     lr █▇▇▆▅▄▄▃▃▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
wandb:       train_loss_batch ▁▁▁▁▁▁▁▁▁█▁▁▁▁▁▁▃▂▃▁▁▁▁▁▁▁▇▇▁▆▁▁▁▁▁▁▁▁▃▁
wandb:     train_loss_element ▁▁▁▁▁▁▁▁▁█▁▁▁▁▁▁▃▂▃▁▁▁▁▁▁▁██▁▆▁▁▁▁▁▁▁▁▃▁
wandb: train_loss_enumeration ▁▁▁▁▁▁▁▁▁█▁▁▁▁▁▁▃▂▃▁▁▁▁▁▁▁▇▇▁▅▁▁▁▁▁▁▂▁▃▁
wandb:        train_loss_site ▁▁▁▁▁▁▁▁▁█▁▁▁▁▁▁▃▂▃▁▁▁▁▁▁▁▇▇▁▆▁▁▁▁▁▁▁▁▃▁
wandb:         val_loss_epoch █▆▄▃▃▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
wandb: 
wandb: Run summary:
wandb:          known_seq_len 20
wandb:                     lr 1e-05
wandb:       train_loss_batch 66.53387
wandb:     train_loss_element 66.53387
wandb: train_loss_enumeration 0.0
wandb:        train_loss_site 0.0
wandb:         val_loss_epoch 9.87184
wandb: 


wandb: 🚀 View run fearless-wind-185 at: https://wandb.ai/kazeev/WyckoffTransformer/runs/9pl53nyu
wandb: ⭐️ View project at: https://wandb.ai/kazeev/WyckoffTransformer
wandb: Synced 5 W&B file(s), 0 media file(s), 0 artifact file(s) and 1 other file(s)


wandb: Find logs at: ./wandb/run-20240517_150429-9pl53nyu/logs
